# ДОМАШНЯЯ РАБОТА - "ВВЕДЕНИЕ В РЕКОМЕНДАТЕЛЬНЫЕ СИСТЕМЫ"

Данильченко Вадим

ЗАДАНИЕ: Постройте топ фильмов в категориях Action и Comedy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [3]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


подтянем к фильмам рейтинги

In [8]:
movie_rating=movies.merge(ratings, on='movieId', how='left')
movie_rating.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08


сформируем 2 датасета - Actions и Comedy

In [22]:
# проверим сколько есть фильмов с обоими жанрами
movie_rating[(movie_rating.genres.str.contains('Comedy')&(movie_rating.genres.str.contains('Action')))]['title'].unique().shape[0]

343

видимо будут пересечения фильмов в жанрах, но думаю это можно опустить в раках решения данного задания

### COMEDY 

In [71]:
# посмотрим на ТОП 10 по средим значениям
ratings_mean=movie_rating[movie_rating.genres.str.contains('Comedy')].groupby('title').rating.mean()
top10_comedy=ratings_mean.sort_values(ascending=False).head(10)
top10_comedy

title
Bill Hicks: Revelations (1993)                                                       5.0
Ivan Vasilievich: Back to the Future (Ivan Vasilievich menyaet professiyu) (1973)    5.0
Choose Me (1984)                                                                     5.0
Schneider vs. Bax (2015)                                                             5.0
Bill Hicks: Relentless (1992)                                                        5.0
Sammy and Rosie Get Laid (1987)                                                      5.0
Grass Is Greener, The (1960)                                                         5.0
The Dress (1996)                                                                     5.0
Saddest Music in the World, The (2003)                                               5.0
Hair (1979)                                                                          5.0
Name: rating, dtype: float64

есть предпосылки считать рейтинг ошибочным, чтобы проверить надо посмотреть на количество оценок по этим фильмам относительно среднего значения количества оценок на фильм в жанре

In [72]:
# количество оценок по топ10 фильмов на основе среднего рейтинга
movie_rating[movie_rating.title.isin(top10_comedy.index)].groupby('title').rating.count()

title
Bill Hicks: Relentless (1992)                                                        1
Bill Hicks: Revelations (1993)                                                       1
Choose Me (1984)                                                                     1
Grass Is Greener, The (1960)                                                         1
Hair (1979)                                                                          1
Ivan Vasilievich: Back to the Future (Ivan Vasilievich menyaet professiyu) (1973)    1
Saddest Music in the World, The (2003)                                               1
Sammy and Rosie Get Laid (1987)                                                      1
Schneider vs. Bax (2015)                                                             1
The Dress (1996)                                                                     1
Name: rating, dtype: int64

In [73]:
# количество оценок на фильм
ratings_cnt=movie_rating[movie_rating.genres.str.contains('Comedy')].groupby('title').rating.count()
ratings_cnt.head()

title
'Hellboy': The Seeds of Creation (2004)     1
'burbs, The (1989)                         19
(500) Days of Summer (2009)                45
*batteries not included (1987)              7
...And God Spoke (1993)                     1
Name: rating, dtype: int64

In [74]:
# среднее количество оценок на фильм в жанре
stat_comedy=ratings_cnt.reset_index().rating.describe()
stat_comedy

count    3315.000000
mean       11.470890
std        23.772991
min         0.000000
25%         1.000000
50%         3.000000
75%        10.000000
max       341.000000
Name: rating, dtype: float64

мы видим, что данные фильмы сожержат по медиане 11 оценок, когда наши топ10 только по одной - рейтинг нельзя считать корректным

посчитаем новый рейтинг, скорректировав рейтинг на отношение количества оценок относительно остальных наблюдений

In [91]:
new_rating_comedy={}
for film in tqdm_notebook(ratings_cnt.index):
    new_rating_comedy.update({film:(ratings_mean[film]*(ratings_cnt[film]-stat_comedy[stat_comedy.index=='mean'][0])/(stat_comedy[stat_comedy.index=='max'][0]-\
        stat_comedy[stat_comedy.index=='min'][0]))})

In [117]:
k=0
for w in sorted(new_rating_comedy, key=new_rating_comedy.get, reverse=True):
    k+=1
    if k<=10:
        print(k,{w: new_rating_comedy[w]})

1 {'Back to the Future (1985)': 2.5262134718399096}
2 {'Ace Ventura: Pet Detective (1994)': 1.3770151290834403}
3 {'Being John Malkovich (1999)': 1.3287893075369357}
4 {'Big Lebowski, The (1998)': 1.271742626970331}
5 {'Breakfast Club, The (1985)': 1.2325880469509138}
6 {'Clerks (1994)': 1.1335407346339925}
7 {'Clueless (1995)': 1.1298485069642565}
8 {'Big (1988)': 1.0734490405702877}
9 {"Bug's Life, A (1998)": 0.9900162751016642}
10 {'Austin Powers: The Spy Who Shagged Me (1999)': 0.9647025254947456}


на мой взгляд отлично получилось - половину фильмов я сам знаю и согласен, что они в топе классики жанра

### ACTION 

In [118]:
# средняя оценка на фильм
ratings_mean_action=movie_rating[movie_rating.genres.str.contains('Action')].groupby('title').rating.mean()

In [119]:
# количество оценок на фильм
ratings_cnt_action=movie_rating[movie_rating.genres.str.contains('Action')].groupby('title').rating.count()
ratings_cnt_action.head()

title
'Hellboy': The Seeds of Creation (2004)                           1
13 Assassins (Jûsan-nin no shikaku) (2010)                        5
13th Warrior, The (1999)                                         25
1990: The Bronx Warriors (1990: I guerrieri del Bronx) (1982)     1
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)            12
Name: rating, dtype: int64

In [121]:
# среднее количество оценок на фильм в жанре
stat_action=ratings_cnt_action.reset_index().rating.describe()
stat_action

count    1544.000000
mean       17.523316
std        33.148121
min         0.000000
25%         1.750000
50%         5.000000
75%        18.000000
max       291.000000
Name: rating, dtype: float64

посчитаем новый рейтинг, скорректировав рейтинг на отношение количества оценок относительно остальных наблюдений

In [123]:
new_rating_action={}
for film in tqdm_notebook(ratings_cnt_action.index):
    new_rating_action.update({film:(ratings_mean_action[film]*(ratings_cnt_action[film]-\
        stat_action[stat_action.index=='mean'][0])/(stat_action[stat_action.index=='max'][0]-\
        stat_action[stat_action.index=='min'][0]))})

In [124]:
k=0
for w in sorted(new_rating_action, key=new_rating_action.get, reverse=True):
    k+=1
    if k<=10:
        print(k,{w: new_rating_action[w]})

1 {'Matrix, The (1999)': 3.471453608866141}
2 {'Jurassic Park (1993)': 3.2665120593330337}
3 {'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)': 2.917599827126562}
4 {'Braveheart (1995)': 2.853496370682953}
5 {'Fight Club (1999)': 2.648741472619566}
6 {'Saving Private Ryan (1998)': 2.351787145735958}
7 {'True Lies (1994)': 2.180078656335267}
8 {'Batman (1989)': 2.0840429115399166}
9 {'Speed (1994)': 1.9914095282184596}
10 {'Blade Runner (1982)': 1.7826344014815978}


и снова в яблочко, хотя с другой стороны странно, что все фильмы ранее 2000г, но это уже дело анализа, возможно фильмы стали менее эффектными или пользователь стал более искушенный, как-нибудь бы проверить..